In [ ]:
# File: twist.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
from frontend import App

# create an app
app = App.create("twist")

# create a cylinder mesh
V, F = app.mesh.cylinder(r=0.5, min_x=-1.0, max_x=1.0, n=360)
app.asset.add.tri("cylinder", V, F)

# create a scene
scene = app.scene.create()

# add cylinder with gradient coloring along axis
obj = scene.add("cylinder").cylinder_color([0, 0, 0], [1, 0, 0], [0, 1, 0])

# set material properties
(
    obj.param.set("friction", 0.0)
    .set("bend", 500)
    .set("contact-gap", 4e-3)
    .set("young-mod", 10000)
)

# define animation parameters for twisting and scaling
angular_vel, move_delta, t_end = 360.0, 0.2, 10
scale = 1 - t_end * 0.01

# pin and animate left end
(
    obj.pin(obj.grab([-1, 0, 0]))
    .spin(axis=[1, 0, 0], angular_velocity=angular_vel)
    .scale(scale, t_start=0.0, t_end=t_end)
    .move_by([move_delta, 0, 0], t_start=0.0, t_end=t_end)
)

# pin and animate right end in opposite direction
(
    obj.pin(obj.grab([1, 0, 0]))
    .spin(axis=[-1, 0, 0], angular_velocity=angular_vel)
    .scale(scale, t_start=0.0, t_end=t_end)
    .move_by([-move_delta, 0, 0], t_start=0.0, t_end=t_end)
)

# compile the scene and report stats
fixed = scene.build().report()

# preview the initial scene
fixed.preview()

In [ ]:
# create a new session with the compiled scene
session = app.session.create(fixed)

# set session parameters - disable gravity and increase sparse matrix capacity
(
    session.param.set("dt", 1 / 60)
    .set("frames", 430)
    .set("gravity", 0.0)
    .set("csrmat-max-nnz", 35000000)
)  # 5GB VRAM needed

# build this session
fixed_session = session.build()

In [ ]:
# start the simulation and live-preview the results
fixed_session.start().preview()

# also show simulation logs in realtime
fixed_session.stream()

In [ ]:
# create an animation from the simulation results
fixed_session.animate()

In [ ]:
# export the animation to file
fixed_session.export.animation()

In [ ]:
# this is for CI
if app.ci:
    assert fixed_session.finished()